In [1]:
# Basics
import numpy as np
import pandas as pd
import os, glob
import pickle

# Keras
import keras.backend as K
import keras.initializers as initializers
from keras.optimizers import Adam, RMSprop, Adadelta
from keras.losses import categorical_crossentropy
from keras.models import Sequential, Model
from keras.layers import Input, Activation, Add, AveragePooling3D
from keras.layers.core import Dense, Dropout, Flatten, Reshape
from keras.layers.convolutional import Conv3D, Cropping3D
from keras.layers.pooling import MaxPooling3D
from keras.layers.wrappers import TimeDistributed, Bidirectional

# Visualization
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model

Using TensorFlow backend.


# Load data partitions

In [2]:
with open("/Users/anders1991/deepbreath/Data/partition.pkl", 'rb') as f:
    partition = pickle.load(f)

In [3]:
np.load("/Users/anders1991/deepbreath/Data/single_train/vol_62.npy").shape

(1, 1, 142, 322, 262)

# Load labels

In [4]:
target = pd.read_csv("/Users/anders1991/deepbreath/Data/ERU_Scores_Ids_5-Scans_Validity-0_VisuallyScored.csv")
labels = target.set_index("StId").to_dict()["ERU.M2"]

# Calculate class weights

To handle imbalanced classes during training.

In [5]:
from sklearn.utils import class_weight

In [6]:
# weights only based on training data
train_labels = [label for key, label in labels.items() if key in partition["train"]]

In [7]:
class_weights = class_weight.compute_class_weight(class_weight='balanced',
                                                  classes=np.unique(train_labels), 
                                                  y=train_labels)
print(class_weights)

[  0.2076226    4.12169312   0.92738095   1.71208791   4.83850932
  18.54761905  55.64285714]


# Set hyperparameters

In [8]:
max_epochs = 2
batch_size = 8
timesteps = 1
learn_rate = 1e-3

# Import data generator

In [9]:
from data_gen import DataGenerator

In [10]:
trainGen = DataGenerator("single_train", batch_size=batch_size, timesteps=timesteps, 
                         channels=1, dim_x=142, dim_y=322, dim_z=262, shuffle=True)
validGen = DataGenerator("single_valid", batch_size=batch_size, timesteps=timesteps, 
                         channels=1, dim_x=142, dim_y=322, dim_z=262, shuffle=False)

trainGen = trainGen.generate(labels, partition["train"])
validGen = validGen.generate(labels, partition["valid"])

# Import model

In [11]:
from models import tdist_unet

In [12]:
model = tdist_unet(timesteps=timesteps)

In [13]:
model.compile(optimizer=Adam(lr = learn_rate), 
              loss='mean_squared_error', 
              metrics=['accuracy'])

# Setup training session

In [ ]:
hist = model.fit_generator(generator = trainGen,
                           steps_per_epoch = len(partition["train"])//batch_size,
                           validation_data = validGen,
                           validation_steps = len(partition["valid"])//batch_size,
                           class_weight = class_weights,
                           epochs = max_epochs,
                           verbose = 2)

Epoch 1/2
